In [1]:

from datetime import datetime
from math import sqrt

from pandas.core.frame import DataFrame
from pandas.core.indexes.datetimes import date_range
from pandas.core.series import Series
from pandas.core.reshape.concat import concat
from pandas.tseries.offsets import BMonthEnd
from scipy.optimize import minimize_scalar
from numpy.linalg.linalg import inv
from scipy.stats import multivariate_normal

# Load Cost Functions Here
from numpy import interp


class CertaintyEquivalentCosts:
    '''
    Assumes Quadratic Utility
    '''

    def __init__(self, risk_aversion, returns, covariances, optimal):
        self.risk_aversion = risk_aversion
        self.returns = returns
        self.covariances = covariances
        self.optimal = optimal

        self.optimal_cer = self.certainty_equivalent_return(optimal)
        self.explicit = False
        self.short_name = 'CEC'

    def verify_inputs(self, index):
        assert (self.returns.index == index).all()
        assert (self.covariances.index == index).all()

    def certainty_equivalent_return(self, portfolio):
        mean = self.returns.mul(portfolio).sum()
        variance = self.covariances.dot(portfolio).dot(portfolio)
        return mean - self.risk_aversion * variance / 2.

    def cost(self, target):
        return self.optimal_cer - self.certainty_equivalent_return(target)

    def dollar_cost(self, target):
        portfolio_size = target.sum()
        scaled_target = target / portfolio_size
        return portfolio_size * self.cost(scaled_target)


class LinearTradingCosts:
    '''
    Bid/Ask Spread ect.
    '''

    def __init__(self, linear_costs):
        self.linear_costs = linear_costs
        self.explicit = True
        self.short_name = 'Trading'

    def verify_inputs(self, index):
        assert (self.linear_costs.index == index).all()

    def cost(self, trades):
        return self.linear_costs.mul(trades.abs()).sum()

    def dollar_cost(self, trades):
        return self.cost(trades)


class EstimatedFutureCosts:
    '''
    Kritzman et. Al. heuristic for future trading costs
    
    http://marriottschool.net/emp/boyer/financeseminar/s07_08/Mark%20Kritzman%20w08.pdf
    '''

    def __init__(self, heuristic, optimal):
        self.heuristic = heuristic
        self.optimal = optimal
        self.explicit = False
        self.short_name = 'Future'

    def verify_inputs(self, index):
        assert (self.optimal.index == index).all()

    def cost(self, target):
        return self.heuristic * (target - self.optimal).mul(target - self.optimal).sum()

    def dollar_cost(self, target):
        portfolio_size = target.sum()
        scaled_target = target / portfolio_size
        return portfolio_size * self.cost(scaled_target)
    
    

In [2]:
from cvxpy import Variable, Parameter, Problem, Minimize, quad_form, abs, sum_squares, sum
from cvxpy.error import SolverError
from pandas.core.series import Series


class TradingProblem:
    def __init__(self, cecs, trading):
        self.cecs = cecs
        self.trading = trading
        
        self.names = [self.cecs.short_name, self.trading.short_name]
    
    def rebalance_costs(self, target, current):
        trades = target - current
        
        cecs = self.cecs.cost(target)
        trading = self.trading.cost(trades)
        return Series([cecs, trading], self.names)


class SimpleConvexTradingSolution(TradingProblem):
    '''
    CVXPY implementation of optimal rebalancing based on Kritzman et. Al:
    
    http://marriottschool.net/emp/boyer/financeseminar/s07_08/Mark%20Kritzman%20w08.pdf
    
    To Use:
    All inputs need to be aligned as in the asserts
    
    set heuristic if you didn't in __init__ (problem.heuristic.value = ...)
    run problem.solve(current_portfolio_in_pandas_series) to get optimal target
        check_no_trade checks to see if not trading at all would be better (0 fixed costs)
    
    for __init__:
    returns, linear_costs, optimal_portfolio, fixed_trades are all Series    
    covariances is a DataFrame
    non_trading_assets, trade_down_assets should be a list of asset names
    out_of_model is a dataframe of weights with out-of-model assets as columns and in-model assets as rows
        out_of_model.sum() == 1 for all assets
    
    '''
    
    def __init__(self, cecs, trading, future):
        super().__init__(cecs, trading)
        self.future = future
        
        # Verify Inputs
        self.assets = self.cecs.returns.index
        self.cecs.verify_inputs(self.assets)
        self.trading.verify_inputs(self.assets)
        self.future.verify_inputs(self.assets)
        
        # CVXPY params
        self.current = Parameter(len(self.assets), nonneg=True, name='Current')
        self.heuristic = Parameter(nonneg=True, name='Heuristic')
        
        self.target = self._build_target()
        self.problem = None  # Lazy initialization
        self.name = 'Optimal Trading'
    
    def update_heuristic(self, new_heuristic):
        self.future.heuristic = new_heuristic
        self.heuristic.value = new_heuristic
        
    def expected_costs(self, target, current, _=None):
        costs = super().rebalance_costs(target, current)
        costs[self.future.short_name] = self.future.cost(target)
        return costs

    def _build_target(self):
        return Variable(len(self.assets), nonneg=True, name='Trading Targets')
    
    def _future_cost_problem(self):
        return self.heuristic * sum_squares(self.target - self.future.optimal.values)
    
    def _trading_cost_problem(self):
        return self.trading.linear_costs.values.T * abs(self.target - self.current)
    
    def _cec_problem(self):
        return self.cecs.optimal_cer - self.cecs.returns.values.T * self.target \
                + self.cecs.risk_aversion * quad_form(self.target, self.cecs.covariances.values) / 2.

    def _build_problem(self):
        # CVXPY cost function setup
        cecs = self._cec_problem()
        t_costs = self._trading_cost_problem()
        f_costs = self._future_cost_problem()
        costs = cecs + t_costs + f_costs
        
        # Constraints
        constraints = [self.target <= 1, sum(self.target) == 1]
        return Problem(Minimize(costs), constraints)
    
    #####  Solver   #######
    def solve(self, current, check_no_trade=True, tol=1.0e-6, warm_start=True, **_):
        if self.problem is None:
            self.problem = self._build_problem()
        
        assert (current.index == self.assets).all()
        self.current.value = current.values
        self.problem.solve(eps_abs=tol, eps_rel=tol, polish=True, warm_start=warm_start)
        if self.problem.status == 'infeasible':
            raise SolverError('Problem Infeasible')
        
        target = Series(self.target.value, self.assets, name='Target')
        target.name = 'Target'
        return target


ModuleNotFoundError: No module named 'cvxpy'

In [9]:
##Krirzman
STANDARD_SEED = 8675309


def random_paths(num_paths, returns, covariances, date_index, period_scale=1, seed=None):
    num_dates = len(date_index)
    period_returns = (1 + returns) ** (1 / period_scale) - 1 if period_scale != 1 else returns
    mn = multivariate_normal(period_returns, covariances / period_scale, allow_singular=True)  # returns / period_scale
    random_returns = mn.rvs(size=num_dates * num_paths, random_state=seed)

    digits = len(str(num_paths))
    keys = [f'Run {str(run_num).zfill(digits)}' for run_num in range(num_paths)]
    return concat([DataFrame(random_returns[i * num_dates:(i + 1) * num_dates], date_index, returns.index) for i in
                   range(num_paths)], axis=1, keys=keys, names=['Run', 'Strategy'])


def implied_returns(weights, returns, var_covar_matrix, risk_aversion, adjust_for_total=True):
    ''' Implied returns given weights/returns that came from a constrained problem'''

    # Standard implied returns
    impl_returns = risk_aversion * var_covar_matrix.dot(weights)

    # Adjustment for sum(x) == 1
    if adjust_for_total:
        inverse_var_covar = DataFrame(inv(var_covar_matrix), var_covar_matrix.index, var_covar_matrix.index)
        impl_returns += (-risk_aversion + inverse_var_covar.dot(returns).sum()) / inverse_var_covar.sum().sum()

    impl_returns.name = 'Implied Returns'
    return impl_returns


def simulate_path(path, trading_solution, start):
    current = start.copy()
    costs = []
    positions = []

    # Turn off warm start at the start of a new run
    warm_start = False
    for date in path.index:
        target = trading_solution.solve(current, warm_start=warm_start, date=date)
        positions.append(target)

        all_costs = trading_solution.rebalance_costs(target, current)
        costs.append(all_costs)

        current = target.mul(1 + path.loc[date])
        current /= current.sum()
        if not warm_start:
            warm_start = True

    positions = DataFrame(positions, path.index)  # TODO Something with positions too
    return DataFrame(costs, path.index)


def figure_of_merit(heuristic, paths, trading_solution, initial_portfolio, verbose=True):
    all_costs = []

    runs = paths.columns.unique('Run')
    trading_solution.update_heuristic(heuristic)
    for path_name in runs:
        path = paths[path_name]
        costs = simulate_path(path, trading_solution, initial_portfolio)
        all_costs.append(costs.sum())

    costs = DataFrame(all_costs).mean()
    if verbose:
        print()
        print(f'heuristic: {heuristic}')
        print(f'Total Cost: {costs.sum() * 10000}')
        print(costs * 10000)  # Costs in bps

    return costs.sum()


def main():
    asof = datetime(2019, 6, 28)
    freq = BMonthEnd()
    periods = 12
    simulation_months = 2 * periods  # Two Years
    simulation_end = asof + simulation_months * freq - freq
    dates = date_range(asof, simulation_end, freq=freq)

    num_paths = 100
    seed = STANDARD_SEED
    risk_aversion = 7.5

    # Optimal Portfolio
    portfolio = Series([60, 40], ['Domestic Equities', 'Domestic Fixed Income']) / 100.
    
    # Trading Cost Assumptions
    linear_costs = Series([40, 45], portfolio.index) / 10000.
    
    # Capital Market Assumptions
    returns = Series([10, 4], portfolio.index) / 100 / 12
    volatilities = Series([12.74, 3.96], portfolio.index) / 100. / sqrt(12)
    correlations = DataFrame([[1, -0.31], [-0.31, 1]], portfolio.index, portfolio.index)
    covariances = correlations.mul(volatilities, axis=1).mul(volatilities, axis=0)

    # Implied Returns
    impl_returns = implied_returns(portfolio, returns, covariances, risk_aversion)
    print(returns * 12 * 100)
    print(impl_returns * 12 * 100)

    # Cost Functions
    cec = CertaintyEquivalentCosts(risk_aversion, impl_returns, covariances, portfolio)
    trading = LinearTradingCosts(linear_costs)
    future = EstimatedFutureCosts(-1, portfolio)  # Heuristic set in figure of merit

    # Set up Problem
    trading_solution = SimpleConvexTradingSolution(cec, trading, future)

    # Build Paths
    paths = random_paths(num_paths, impl_returns, covariances, dates, seed=seed)

    # Find Heuristic
    args = (paths, trading_solution, portfolio)
    min_output = minimize_scalar(figure_of_merit, bounds=(0.001, 0.1), args=args, method='bounded')
    print(f'Minimum Cost {min_output}')
    d = min_output.x
    print(f'Optimal Heuristic {d}')


if __name__ == '__main__':
    main()


Domestic Equities        10.0
Domestic Fixed Income     4.0
dtype: float64
Domestic Equities        10.908142
Domestic Fixed Income     3.840154
Name: Implied Returns, dtype: float64


NameError: name 'SimpleConvexTradingSolution' is not defined

In [11]:
# Optimal Portfolio
    portfolio = Series([60, 40], ['Domestic Equities', 'Domestic Fixed Income']) / 100.
    
    # Trading Cost Assumptions
    linear_costs = Series([40, 45], portfolio.index) / 10000.

IndentationError: unexpected indent (<ipython-input-11-4125a86dffb0>, line 2)